In [8]:
import pandas as pd

from utils import tag_mail

df = pd.read_json(r'data/gmail_azoulai_basic.txt', encoding = 'utf-8')

df['cat'] = df.headers.apply(tag_mail)

df.loc[df.cat=='uber']

,snippet,date,headers,body,cat
73,"Total: 2,65 € 1 février 2020 Merci d&#39;avoir...",2020-02-01 20:56:10,"[{'name': 'From', 'value': '""Reçu Uber"" <recei...","Total: 2,65 € 1 février 2020 Merci d'avoir ch...",uber
117,"Total: 9,19 € sam., janv. 18, 2020 Paul, merci...",2020-01-18 02:07:59,"[{'name': 'Subject', 'value': '[Personal] Votr...",,uber
176,"Total: 2,20 € 4 janvier 2020 Merci d&#39;avoir...",2020-01-04 19:04:33,"[{'name': 'From', 'value': '""Reçu Uber"" <recei...","Total: 2,20 € 4 janvier 2020 Merci d'avoir ch...",uber
252,"Total: 5,26 € jeu., déc. 05, 2019 Paul, merci ...",2019-12-05 01:19:49,"[{'name': 'From', 'value': '""Reçu Uber"" <uber....",,uber
261,"Total: 15,96 € sam., nov. 30, 2019 Paul, merci...",2019-11-30 01:12:05,"[{'name': 'From', 'value': '""Reçu Uber"" <uber....",,uber
313,"Total: 6,83 € dim., nov. 03, 2019 Paul, merci ...",2019-11-03 01:25:30,"[{'name': 'From', 'value': '""Reçu Uber"" <uber....",,uber
314,Créez automatiquement des notes de frais pour ...,2019-11-03 01:00:25,"[{'name': 'From', 'value': 'Uber <no-reply@ube...",Créez automatiquement des notes de frais pour...,uber
454,1 plat acheté = 1 plat offert ! Qui allez-vous...,2019-10-01 17:05:22,"[{'name': 'From', 'value': 'Uber Eats <uber@ub...",1 plat acheté = 1 plat offert ! QUI ALLEZ-VOUS...,uber
569,"Total: 6,47 € mar., sept. 03, 2019 Paul, merci...",2019-09-03 01:15:42,"[{'name': 'From', 'value': '""Reçu Uber"" <uber....",,uber
570,On te partage les bons plans UberPool x Vice T...,2019-09-03 01:15:32,"[{'name': 'From', 'value': 'Uber <uber@uber.co...",On te partage les bons plans UberPool x ViceTU...,uber


In [15]:
import re
from datetime import datetime

df_uber = df.loc[df.cat=='uber']

df_uber = df_uber.loc[df_uber.body.str.contains("vélos électriques")]

res = []

for date, el in df_uber.loc[:,['date', 'body']].itertuples(index=False):
    tmp = {}
    # place
    try:
        departure, destination = re.findall(r'\d\d:\d\d (.*?) \d\d:\d\d (.*?) contacter', el)[0]
        tmp['departure'] = departure
        tmp['destination'] = destination
    except:
        continue
    
    # price    
    try:
        price = re.findall(r' Total: \d{1,1000},\d{2,1000} € ',el )[0]
        price = re.findall(r'\d{1,1000},\d{2,1000}', price)[0]
        tmp['price'] = price
    except:
        price = None
        tmp['price'] = price
        pass
    
    # distance
    try:
        distance = re.findall(r'\d{1,1000}.{0,1}\d{0,1000} kilomètres', el)[0]
        tmp['distance'] = distance
    except:
        distance = None
        tmp['distance'] = distance
        pass
    
    # date
    try:
        horaire = re.findall(r'\d\d:\d\d', el)
        start = datetime.strptime(horaire[0], '%H:%M')
        end = datetime.strptime(horaire[1], '%H:%M')
        start = start.replace(year = date.year, month = date.month, day = date.day)
        end = end.replace(year = date.year, month = date.month, day = date.day)
        tmp['start'] = start
        tmp['end'] = end
    except:
        start = None
        end = None
        tmp['start'] = start
        tmp['end'] = end
        pass
    
    res += [tmp]
    print(el)
    print('\n\n')
    print('#######################################################')
    print('\n\n')
    
res

 Total: 2,65 € 1 février 2020 Merci d'avoir choisi les vélos électriques JUMP, Paul Nous espérons que vous avez apprécié votre course ce soir. Total 2,65 € Prix de base 1,00 € Prix pour la durée de la course 1,65 € Sous-total 2,65 € Montant facturé •••• 4130 2,65 € Pour consulter vos reçus (et vos factures le cas échéant), ou pour obtenir plus d'informations, rendez-vous sur votre espace personnel. Vous avez utilisé le vélo électrique JUMP numéro WMM513. 1.61 kilomètres | 11 min 21:45 2 Impasse du Dr Jacques Bertillon, 75008 Paris, France 21:56 20 Avenue des Ternes, 75017 Paris, France contacter l'assistance ❯ contacter l'assistance ❯ Mot de passe oublié Uber B.V.
Mr. Treublaan 7
1097 DP Amsterdam Confidentialité 
Conditions générales Le prix n'inclut pas d'éventuels frais bancaires. Contactez votre banque pour en savoir plus sur les frais applicables. 



#######################################################



 Total: 2,20 € 4 janvier 2020 Merci d'avoir choisi les vélos électriques

[{'departure': '2 Impasse du Dr Jacques Bertillon, 75008 Paris, France',
  'destination': '20 Avenue des Ternes, 75017 Paris, France',
  'price': '2,65',
  'distance': '1.61 kilomètres',
  'start': datetime.datetime(2020, 2, 1, 21, 45),
  'end': datetime.datetime(2020, 2, 1, 21, 56)},
 {'departure': '43 Rue François Gérard, 75016 Paris, France',
  'destination': 'marché de Passy coté 17 Rue Duban, 75016 Paris, France',
  'price': '2,20',
  'distance': '1.66 kilomètres',
  'start': datetime.datetime(2020, 1, 4, 19, 56),
  'end': datetime.datetime(2020, 1, 4, 20, 4)}]

In [3]:
df_uber.loc[738].body

" Total: 74,92 € mar., déc. 10, 2019 Etienne, merci d'avoir utilisé Uber Nous espérons que vous avez apprécié votre course cet après-midi. Total 74,92 € Prix du trajet 74,92 € Sous-total 74,92 € Montant facturé Bon : Uber rides for Etienne Turc 74,92 € Pour consulter vos reçus (et vos factures le cas échéant), ou pour obtenir plus d'informations, rendez-vous sur votre espace personnel. xidb5307955-890f-4898-9f52-0c5555da82d7 pGvlI2ANUbXFfyEOgxta1RMV082993 Votre chauffeur était Hafedh 4.91 Note Hafedh est apprécié pour : \nExcellent Service Comment s'est passée votre course ? Note ou pourboire Note ou pourboire Reçu émis au nom de DH TRANSPORT DE PERSONNES UberX 58.08 km | 1 h 11 min 14:32 93290 Tremblay-en-France, France 15:43 21 Rue Maximilien Robespierre, 91120 Palaiseau, France Invitez vos amis et votre famille. Bénéficiez d’une réduction de 5 € sur votre prochaine course en invitant un ami à découvrir Uber.Code de parrainage : 01i3pmSignaler la perte d'un objet ❯contacter l'assista

In [41]:
t = ' Total: 74,92 € '

In [43]:
re.findall('\d{1,1000},\d{2,1000}', t)[0]

'74,92'

In [85]:
for date, mail in df_uber.loc[:,['date', 'body']].itertuples(index=False):
    print(date)

2019-12-10 14:43:37
2019-12-08 09:58:55
2018-11-23 18:41:55


In [82]:
for a, b in df_uber.itertuples(index=False):
    print(a)

ValueError: too many values to unpack (expected 2)